In [1]:
import numpy as np
import ckwrap
from normals_init import find_neighs
from joblib import Parallel, delayed
import os
import torch

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
all_pts = np.load("xyz_points.npy")
all_norms = np.load("init_normals.npy")

In [3]:
'''creating patches via clustering'''

def process_one_pcd(pts, norms, main_ind):
    dists_ori, indices_ori = find_neighs(pts)
    neighs = pts[indices_ori]
    print("neighs shape: ", neighs.shape)
    labels_per_row = np.apply_along_axis(lambda row: ckwrap.ckmeans(row, 5).labels, 1, dists_ori)
    print("labels shape: ",labels_per_row.shape)

    pts = torch.from_numpy(pts)
    norms = torch.from_numpy(norms)
    indices_ori = torch.from_numpy(indices_ori)
    labels_per_row = torch.from_numpy(labels_per_row)

    for i,( ind, labels) in enumerate(zip(indices_ori, labels_per_row)):
        center = pts[i]
        print("labels shape:", labels.shape)
        for clus in torch.unique(labels, dim=0):
            # ind_clus = ind[np.where(labels == clus)]
            ind_clus = ind[torch.where(labels==clus)]
            print("ind_clus shape: ",ind_clus.shape)
            coords = pts[ind_clus]
            print("coords shape: ", coords.shape)

            if(coords.shape[0] < 25):
                # 2 random integers
                for i in range(25-(coords.shape[0])):
                    a, b = torch.randint(1, coords.shape[0], (2,))
                    a_n, b_n = torch.divide(a, (a+b)), torch.divide(b, (a+b))
                    rnd_pt = torch.add(torch.multiply(a_n, (center-coords[a])), torch.multiply(b_n, (center-coords[b]))).unsqueeze(0)
                    coords = torch.cat((coords, rnd_pt), dim=0)
            else:
                coords = coords[:25]
            
            print("coords after adding random points: ", coords)

            vec2 = torch.subtract(center,coords)
            print("vec2 : ", vec2)
            print("nearest vec magnitude: ", torch.linalg.norm(vec2, dim=1))
            closest_ind = torch.argmin(torch.linalg.norm(vec2, dim=1))
            closest = coords[closest_ind]
            # vec2 = np.delete(coords, closest_ind, axis=0)
            vec2 = torch.cat((coords[:closest_ind], coords[closest_ind+1:]), dim=0)
            vec1 = torch.subtract(center, closest)
            print(vec1.shape)
            vec1 = torch.divide(vec1, torch.linalg.norm(vec1))
            vec2 = torch.divide(vec2, torch.linalg.norm(vec2, dim=1, keepdim=True))
            print("after norm: ", vec2.shape, vec1.shape)
            print(vec2)
            
            crs_pdts = torch.cross(vec1.expand_as(vec2), vec2) #check done, works as intented
            area = torch.linalg.norm(crs_pdts, dim=1)
            crs_pdts = torch.divide(crs_pdts, torch.linalg.norm(crs_pdts, dim=1, keepdim=True))
            print("crs_pdts after norm shape: ",crs_pdts.shape)
            
            print("shapes for dot: ", crs_pdts.shape, norms[i].shape)
            dot_pdts = torch.tensordot(crs_pdts, norms[i].T, dims=1) #check done, works as intented
            sign = torch.divide(dot_pdts, np.abs(dot_pdts))
            print("dot_pdts shape: ",dot_pdts.shape)
            print("area shape: ",area.shape)
            area = torch.multiply(area, sign)
            sort_indices = torch.argsort(area)
            print("area shape after multi with sign: ",area.shape)
            # area, orient, ind_clus = area[sort_indices], orient[sort_indices], ind_clus[sort_indices]
            # coords_sorted= pts[ind_clus]
            sorted_coords = coords[sort_indices]
            
            return dot_pdts, area, crs_pdts, coords, sorted_coords, vec1, vec2, sign, closest, center
        #     mat.append(sorted_coords)
        # mat = np.array(mat)
        # np.save("patches/pcd_"+str(main_ind)+"/pnt_"+str(i)+".npy", mat)
            # break
        break


dot_pdts, area, crs_pdts, coords, sorted_coords, vec1, vec2, sign, closest, center = process_one_pcd(all_pts[0], all_norms[0], 0)

neighs shape:  (100000, 249, 3)
labels shape:  (100000, 249)
labels shape: torch.Size([249])
ind_clus shape:  torch.Size([23])
coords shape:  torch.Size([23, 3])
coords after adding random points:  tensor([[-1.0000e+00,  9.0934e-02,  1.1721e+01],
        [-1.0000e+00,  1.0145e-01,  1.1705e+01],
        [-1.0000e+00,  1.1260e-01,  1.1715e+01],
        [-1.0000e+00,  7.9621e-02,  1.1731e+01],
        [-1.0000e+00,  7.5980e-02,  1.1725e+01],
        [-1.0000e+00,  8.0252e-02,  1.1704e+01],
        [-1.0000e+00,  1.1485e-01,  1.1709e+01],
        [-1.0000e+00,  9.9300e-02,  1.1695e+01],
        [-1.0000e+00,  1.0221e-01,  1.1688e+01],
        [-1.0000e+00,  7.8690e-02,  1.1748e+01],
        [-1.0000e+00,  6.4145e-02,  1.1736e+01],
        [-1.0000e+00,  8.7351e-02,  1.1753e+01],
        [-1.0000e+00,  5.8686e-02,  1.1715e+01],
        [-1.0000e+00,  8.8460e-02,  1.1756e+01],
        [-1.0000e+00,  6.8877e-02,  1.1747e+01],
        [-1.0000e+00,  6.3158e-02,  1.1694e+01],
        [-1.0000e+

/tmp/ipykernel_155082/104509350.py:57: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1666642975993/work/aten/src/ATen/native/TensorShape.cpp:3277.)
  dot_pdts = torch.tensordot(crs_pdts, norms[i].T, dims=1) #check done, works as intented


In [44]:
'''creating patches via clustering'''

def process_one_pcd(pts, norms, main_ind):
    dists_ori, indices_ori = find_neighs(pts)
    neighs = pts[indices_ori]
    print("neighs shape: ", neighs.shape)
    labels_per_row = np.apply_along_axis(lambda row: ckwrap.ckmeans(row, 5).labels, 1, dists_ori)
    print("labels shape: ",labels_per_row.shape)

    pts = torch.from_numpy(pts)
    norms = torch.from_numpy(norms)
    indices_ori = torch.from_numpy(indices_ori)
    labels_per_row = torch.from_numpy(labels_per_row)

    for i,( ind, labels) in enumerate(zip(indices_ori, labels_per_row)):
        print(i)
        center = pts[i]
        print("labels shape:", labels.shape)
        for clus in torch.unique(labels, dim=0):
            # ind_clus = ind[np.where(labels == clus)]
            ind_clus = ind[torch.where(labels==clus)]
            print("ind_clus shape: ",ind_clus.shape)
            coords = pts[ind_clus]
            print("coords shape: ", coords.shape)

            if(coords.shape[0] < 25):
                # 2 random integers
                for _ in range(25-(coords.shape[0])):
                    a, b = torch.randint(1, coords.shape[0], (2,))
                    a_n, b_n = torch.divide(a, (a+b)), torch.divide(b, (a+b))
                    rnd_pt = torch.add(torch.multiply(a_n, (center-coords[a])), torch.multiply(b_n, (center-coords[b]))).unsqueeze(0)
                    coords = torch.cat((coords, rnd_pt), dim=0)
            else:
                coords = coords[:25]
            
            # print("coords after adding random points: ", coords)

            
            vec2 = torch.subtract(center, coords)
            print("magnitudes: ", torch.linalg.norm(vec2, dim=1))
            closest_ind = torch.argmin(torch.linalg.norm(vec2, dim=1))
            closest = coords[closest_ind]
            vec1 = torch.subtract(center, closest)
            vec2 = torch.cat((coords[:closest_ind], coords[closest_ind+1:]), dim=0)
            print(vec1, vec2[2,:])
            vec1 = torch.divide(vec1, torch.linalg.norm(vec1))
            vec2 = torch.divide(vec2, torch.linalg.norm(vec2, dim=1, keepdim=True))
            print(vec1, vec2[0,:])
            crs_pdts = torch.cross(vec1.expand_as(vec2), vec2)
            print(crs_pdts[2,:])
            area = torch.linalg.norm(crs_pdts, dim=1)
            crs_pdts = torch.divide(crs_pdts, torch.linalg.norm(crs_pdts, dim=1, keepdim=True))
            print(crs_pdts[2,:])
            print("norms check: ", i, norms[i].shape)
            dot_pdts = torch.tensordot(crs_pdts, norms[i].T, dims=1)
            print(dot_pdts[2])
            sign = torch.divide(dot_pdts, np.abs(dot_pdts))
            print("dot_pdts shape: ",dot_pdts.shape)
            print("area shape: ",area.shape)
            area = torch.multiply(area, sign)
            sort_indices = torch.argsort(area)
            print("area shape after multi with sign: ",area.shape)
            # area, orient, ind_clus = area[sort_indices], orient[sort_indices], ind_clus[sort_indices]
            # coords_sorted= pts[ind_clus]
            sorted_coords = coords[sort_indices]
            
            return dot_pdts, area, crs_pdts, coords, sorted_coords, vec1, vec2, sign, closest, center
        #     mat.append(sorted_coords)
        # mat = np.array(mat)
        # np.save("patches/pcd_"+str(main_ind)+"/pnt_"+str(i)+".npy", mat)
            # break
        break


dot_pdts, area, crs_pdts, coords, sorted_coords, vec1, vec2, sign, closest, center = process_one_pcd(all_pts[0], all_norms[0], 0)

neighs shape:  (100000, 249, 3)
labels shape:  (100000, 249)
0
labels shape: torch.Size([249])
ind_clus shape:  torch.Size([23])
coords shape:  torch.Size([23, 3])
magnitudes:  tensor([4.4958e-03, 1.5366e-02, 1.8454e-02, 1.9165e-02, 1.9586e-02, 2.0308e-02,
        2.2338e-02, 2.3485e-02, 3.1210e-02, 3.3688e-02, 3.4890e-02, 3.5255e-02,
        3.5878e-02, 3.8078e-02, 3.8627e-02, 3.9318e-02, 3.9370e-02, 4.0802e-02,
        4.3981e-02, 4.4165e-02, 4.5709e-02, 4.7129e-02, 5.1647e-02, 1.1777e+01,
        1.1774e+01], dtype=torch.float64)
tensor([ 0.0000,  0.0035, -0.0029], dtype=torch.float64) tensor([-1.0000,  0.0796, 11.7305], dtype=torch.float64)
tensor([ 0.0000,  0.7681, -0.6404], dtype=torch.float64) tensor([-0.0851,  0.0086,  0.9963], dtype=torch.float64)
tensor([0.7696, 0.0544, 0.0652], dtype=torch.float64)
tensor([0.9940, 0.0703, 0.0843], dtype=torch.float64)
norms check:  0 torch.Size([3])
tensor(0.9833, dtype=torch.float64)
dot_pdts shape:  torch.Size([24])
area shape:  torch.Size

In [43]:
area

tensor([0.7755, 0.7760, 0.7743, 0.7741, 0.7743, 0.7762, 0.7753, 0.7755, 0.7742,
        0.7734, 0.7747, 0.7732, 0.7747, 0.7737, 0.7734, 0.7772, 0.7736, 0.7771,
        0.7769, 0.7726, 0.7736, 0.7736, 0.1832, 0.1997], dtype=torch.float64)

In [42]:
sign

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.], dtype=torch.float64)

In [45]:
dot_pdts

tensor([ 0.9833,  0.9833,  0.9833,  0.9833,  0.9832,  0.9833,  0.9833,  0.9833,
         0.9833,  0.9833,  0.9833,  0.9832,  0.9833,  0.9833,  0.9832,  0.9833,
         0.9833,  0.9833,  0.9833,  0.9832,  0.9833,  0.9833, -0.9956,  0.9956],
       dtype=torch.float64)

In [27]:
# print(coords.shape)
# print(closest, all_pts[0][0])

vec1 = torch.subtract(center, closest)
vec2 = torch.subtract(center, coords[2])
print(vec1, vec2)
vec1 = torch.divide(vec1, torch.linalg.norm(vec1))
vec2 = torch.divide(vec2, torch.linalg.norm(vec2))
print(vec1, vec2)
crs = torch.cross(vec1, vec2)
print(crs)
crs = torch.divide(crs, torch.linalg.norm(crs))
print(crs)
dot = torch.dot(crs, torch.tensor(all_norms[0][0]))
print(dot)

tensor([ 0.0000,  0.0035, -0.0029], dtype=torch.float64) tensor([ 0.0000, -0.0182,  0.0030], dtype=torch.float64)
tensor([ 0.0000,  0.7681, -0.6404], dtype=torch.float64) tensor([ 0.0000, -0.9870,  0.1607], dtype=torch.float64)
tensor([-0.5087, -0.0000, -0.0000], dtype=torch.float64)
tensor([-1., -0., -0.], dtype=torch.float64)
tensor(-0.9956, dtype=torch.float64)


In [30]:
vec1 = torch.subtract(center, closest)
vec2 = torch.subtract(center, coords)
print(vec1, vec2[2,:])
vec1 = torch.divide(vec1, torch.linalg.norm(vec1))
vec2 = torch.divide(vec2, torch.linalg.norm(vec2, dim=1, keepdim=True))
print(vec1, vec2[0,:])
crs = torch.cross(vec1.expand_as(vec2), vec2)
print(crs[2,:])
crs = torch.divide(crs, torch.linalg.norm(crs, dim=1, keepdim=True))
print(crs[2,:])
dots = torch.tensordot(crs, torch.tensor(all_norms[0][0]).T, dims=1)
print(dots[2])

tensor([ 0.0000,  0.0035, -0.0029], dtype=torch.float64) tensor([ 0.0000, -0.0182,  0.0030], dtype=torch.float64)
tensor([ 0.0000,  0.7681, -0.6404], dtype=torch.float64) tensor([ 0.0000,  0.7681, -0.6404], dtype=torch.float64)
tensor([-0.5087, -0.0000, -0.0000], dtype=torch.float64)
tensor([-1., -0., -0.], dtype=torch.float64)
tensor(-0.9956, dtype=torch.float64)


In [37]:
norms = all_norms[0]
dots = torch.tensordot(crs, torch.tensor(norms[0]).T, dims=1)
print(dots[2])

tensor(-0.9956, dtype=torch.float64)
